In [17]:
import numpy as np
import os 
import torch
import torch
import pandas as pd
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import Dataset
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import json

In [3]:
xdim = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11}
ydim = [i for i in range(19, 0, -1)]
# print(xdim)
# print(ydim)
def get_hold_coords(hold, xdim = xdim, ydim = ydim):
    col = xdim[hold[0]] -1
    row = ydim[int(hold[1:])] - 1
    return (row, col)

In [4]:
moonboard_2016_filename = "../data/mb_problem_holds.csv"
moonboard_2016_size = (18,11)
df = pd.read_csv(moonboard_2016_filename)
print(len(df))
climb = df.iloc[10]
print(climb)
print(climb['Moves'])


542
Method                                                      Feet follow hands
Name                                                         ONE ARMED BANDIT
Grade                                                                      8A
UserGrade                                                                  8A
MoonBoardConfiguration                                                    NaN
MoonBoardConfigurationId                                                    0
Setter                      {'Id': '89b67a5a-62d2-46df-95d6-4a5e25b0c361',...
FirstAscender                                                           False
Rating                                                                      0
UserRating                                                                  5
Repeats                                                                     0
Attempts                                                                    0
Holdsetup                   {'Id': 1, 'Description': 'MoonBo

In [14]:
# #Split Data
# def split_csv_and_save(input_file, output_dir, train_size=0.8, random_state=None):
#     # Create the output directory if it doesn't exist
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)
    
#     # Read the CSV file
#     data = pd.read_csv(input_file)
    
#     # Split the data into train and test sets
#     train_data, test_data = train_test_split(data, train_size=train_size, random_state=random_state)
    
#     # Define the paths for train and test files
#     train_file = os.path.join(output_dir, "train.csv")
#     test_file = os.path.join(output_dir, "test.csv")
    
#     # Save train and test data to CSV files
#     train_data.to_csv(train_file, index=False)
#     test_data.to_csv(test_file, index=False)

# # Example usage
# moonboard_2016_filename = "../data/mb_problem_holds.csv"
# output_directory = "../modeldata/splits"
# split_csv_and_save(moonboard_2016_filename, output_directory, train_size=0.8, random_state=42)

In [24]:
french_grade_order = ['4', '5', '5+', '6A', '6A+', '6B', '6B+', '6C', '6C+', '7A', '7A+',
                       '7B', '7B+', '7C', '7C+', '8A', '8A+', '8B', '8B+']
grade_bounds = ('6B+', '8A')
scale_min = french_grade_order.index(grade_bounds[0])
scale_max = french_grade_order.index(grade_bounds[1])
scale_bounds = (scale_min, scale_max)
scale_size = scale_max - scale_min

class ClimbDataset(Dataset):
    def __init__(self, climbs_filename, board_size, scale_bounds):
        self.climb_df = pd.read_csv(climbs_filename)
        self.board_size = board_size
        self.scale_min = scale_bounds[0]
        self.scale_max = scale_bounds[1]
        self.scale_size = self.scale_max - self.scale_min
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        climb = self.climb_df.iloc[idx]
        grade = climb['Grade']
        moves = climb['Moves']
        moves = moves.replace("'", '"')
        moves = moves.replace("True", 'true')
        moves = moves.replace("False", 'false')
        moves = json.loads(moves)
        holds = np.zeros(self.board_size)
        for hold in moves:
            x, y = get_hold_coords(hold['Description'])
            holds[x, y] = 1
        curr_grade_ind = french_grade_order.index(grade)
        if self.scale_min > curr_grade_ind :
            grade_num = 0
        elif self.scale_max < curr_grade_ind :
            grade_num = self.scale_max - self.scale_min
        else:
            grade_num = curr_grade_ind - self.scale_min

        return holds, grade_num

train_dataset = ClimbDataset("../modeldata/splits/train.csv", moonboard_2016_size, scale_bounds)
test_dataset = ClimbDataset("../modeldata/splits/test.csv", moonboard_2016_size, scale_bounds)
train_dataset.__getitem__(5)

(array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 2)

In [25]:
import torch.nn as nn
import torch.nn.functional as F


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

input_size = moonboard_2016_size[0] * moonboard_2016_size[1]
output_size = scale_size
class NeuralNetwork_Orig(nn.Module):
    def __init__(self):
        super(NeuralNetwork_Orig, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, scale_size),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

SyntaxError: invalid syntax (1503634655.py, line 9)